In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [7]:
from langchain.document_loaders import WebBaseLoader

URLS = [
    'https://lscosmetics.ma/',
    'https://baraboucle.com/blogs/journal/quel-produit-pour-mon-enfant',
    'https://www.lexpress.fr/styles/beaute/ma-fille-hurle-avant-meme-que-la-brosse-atteigne-ses-cheveux_1243047.html',
    'https://www.ducray.com/fr-be/etats-pelliculaires/symptomes/pellicules-enfant',
    'https://dreamact.eu/fr/blog/article/162/huiles-vegetales',
    'https://aunea-cosmetique.fr/comment-prendre-soin-des-cheveux-crepus-des-enfants-au-quotidien/',
    'https://lilikiwi.fr/fr/blog/comment-choisir-un-shampoing-pour-enfants--n25',
    'https://cheveux.ooreka.fr/astuce/voir/645153/proteger-vos-cheveux-de-la-piscine',
    'https://www.passeportsante.net/fr/Actualites/Dossiers/DossierComplexe.aspx?doc=10-astuces-pour-eviter-cheveux-electriques',
    'https://madame.lefigaro.fr/beaute/ces-erreurs-que-lon-fait-tous-avec-notre-shampoing-071220-194052',
    'https://www.nuoobox.com/blogs/conseils/cosmetiques-bio-conventionnels-differences',
    'https://www.kryneo.com/blogs/conseils-capillaires/les-7-types-d-ingredients-a-eviter-dans-les-produits-capillaires',
    'https://volna-cosmetiques.fr/routine-cheveux-enfants-ideale/',
    'https://clean-sponge.fr/blogs/soin-des-cheveux/conseils-pour-choisir-des-produits-capillaires-vegan-et-ecologiques'
]

loader = WebBaseLoader(URLS)
data = loader.load()

In [8]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
websites_data = text_splitter.split_documents(data)

Created a chunk of size 10489, which is longer than the specified 5000


In [9]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(deployment="embeddings-cosmetics", embedding_ctx_length=1000)

In [10]:
from langchain.vectorstores import Chroma

websearch = Chroma.from_documents(websites_data, embeddings)

In [11]:
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model='gpt-4',temperature=0)

In [12]:
from langchain.chains import RetrievalQA
from langchain.memory import ConversationSummaryMemory
memory = ConversationSummaryMemory(
    llm=model, memory_key="chat_history"
)

rag = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=websearch.as_retriever())

question = "Comment gérer les cheveux bouclés de mon enfant chaque matin?"
response = rag.run(question)
print(response)

Pour gérer les cheveux bouclés de votre enfant chaque matin, voici quelques conseils :

1. Évitez les brosses : Brosser des cheveux bouclés peut les casser et être douloureux pour votre enfant. Privilégiez un peigne à dents larges ou encore mieux, vos doigts.

2. Démêlez et hydratez régulièrement : Pour éviter une trop longue session hebdomadaire de démêlage, essayez de coiffer les cheveux de votre enfant tous les jours si votre emploi du temps le permet. Pour aider les nœuds à se défaire et hydrater les longueurs, appliquez une crème ou un lait hydratant sur cheveux secs avant de commencer à démêler.

3. Utilisez des produits adaptés : Si les cheveux de votre enfant sont plutôt secs, utilisez la crème Magic Twist des Secrets de Loly, spécialement conçue pour les enfants. Si votre enfant a plutôt des boucles lâches, privilégiez le Smoothie Capillaire des Secrets de Loly, plus léger mais tout aussi hydratant.

4. Chaque matin, sur les cheveux encore mouillés, appliquez un peu de Smoothi

In [14]:
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationSummaryMemory(
    llm=model, memory_key="chat_history")

template = """You are an assistant for question-answering tasks in the field of cosmetics,
Use the following pieces of context to answer the question at the end. 
You should answer the user in his language.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Take into consideration the Product Highlights:Pack Enfant: Essential for children's curly haircare.
Gel de lin: For defining and structuring hair.
Leave in: For deep moisturization.
Après-shampoing ultra démêlant: The ultimate solution for detangling.
Sérum cheveux sans rinçage à l’acide hyaluronique: For daily hair management
Don't forget to ask the user in the right moment about his contact.
Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""

rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": websearch.as_retriever() , "question": RunnablePassthrough()}
    | rag_prompt_custom
    | model
    | StrOutputParser()
)


In [15]:
rag_chain.invoke("Pouvez-vous me donner plus de détails sur le Pack Enfant?")

"Le Pack Enfant est essentiel pour le soin des cheveux bouclés des enfants. Il comprend plusieurs produits qui aident à définir, structurer et hydrater les cheveux. Il contient également un après-shampoing ultra démêlant qui est la solution ultime pour le démêlage des cheveux. Enfin, le pack comprend un sérum pour cheveux sans rinçage à l’acide hyaluronique qui est utile pour la gestion quotidienne des cheveux. Merci de me fournir votre contact pour plus de détails. Merci de m'avoir posé la question!"

In [ ]:
!chainlit run app.py -w